# Convert TEI into TSV


In [14]:
import re
import os
from lxml import etree
import pandas as pd

In [ ]:
inputpath = os.path.join('..', 'data', '1743_LeRobert', 'tei')

data = []

for filename in sorted(os.listdir(inputpath)):
    try:
        parser = etree.XMLParser(collect_ids=False, encoding='utf-8') 
        root = etree.parse(os.path.join(inputpath, filename), parser=parser).getroot()    
        #print(root.nsmap)
        print(filename)
        volume = filename[2:3]

        for entry in root.findall('.//entry', namespaces=root.nsmap):
            entry_id = entry.get('id')
            form = entry.find('.//form[@type="lemma"]/orth', namespaces=root.nsmap)
            if form is not None:
                entry_lemma = form.text
            else:
                print("Forme : non trouvée")
                entry_lemma = None

            for i, subordinate in enumerate(entry.findall('.//sense', namespaces=root.nsmap)):
                
                form = subordinate.find('.//form/orth', namespaces=root.nsmap)
                if form is not None:
                    subordinate_lemma = form.text
                else:
                    subordinate_lemma = None

                usg = subordinate.find('.//usg[@type="domain"]', namespaces=root.nsmap)
                if usg is not None:
                    subordinate_domain = usg.text
                else:
                    subordinate_domain = None

                #TODO ajout du texte de la définition
                #....

                row = [volume, entry_id, entry_lemma, i+1, subordinate_lemma, subordinate_domain]
                #volume | entry | entry_lemma | subordinate | subordinate_lemma | subordinate_domain | content
                data.append(row)
                
                #print(row)
        
    except etree.XMLSyntaxError as e:
        print(f"Erreur de syntaxe XML : {e}")



# convert data into a dataframe

df = pd.DataFrame(data, columns=['volume', 'entry', 'entry_lemma', 'subordinate', 'subordinate_lemma', 'subordinate_domain'])
df.head()


TR1.tei
Forme : non trouvée
Forme : non trouvée
Forme : non trouvée
Forme : non trouvée
TR2.tei
TR3.tei
Forme : non trouvée
TR4.tei
Forme : non trouvée
Forme : non trouvée
Forme : non trouvée
TR5.tei
TR6.tei


,volume,entry,entry_lemma,subordinate,subordinate_lemma,subordinate_domain
0,1,250000010,A,1,None,None
1,1,250000010,A,2,None,None
2,1,250000010,A,3,None,None
3,1,250000010,A,4,None,None
4,1,250000010,A,5,None,None


In [22]:
df.to_csv(os.path.join('..', 'data', '1743_LeRobert', 'Trevoux1743.tsv'), sep='\t', index=False, encoding='utf-8')

In [17]:
df.shape

(135004, 6)